In [1]:
import pandas as pd
import matplotlib as plt
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# Import our input dataset
charityDF = pd.read_csv('charity_data.csv')
charityDF.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
charityDF.STATUS.value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [4]:
charityDF.SPECIAL_CONSIDERATIONS.value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [5]:
# Remove EIN, NAME, STATUS and SPECIAL_CONSIDERATIONS from Dataset, not usefull for analysis
charityDF = charityDF.drop(["EIN","NAME", "STATUS", "SPECIAL_CONSIDERATIONS"],axis = 1)
charityDF.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [6]:
# Generate our categorical variable list
charity_cat = charityDF.dtypes[charityDF.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charityDF[charity_cat].nunique()

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
USE_CASE             5
ORGANIZATION         4
INCOME_AMT           9
dtype: int64

In [7]:
# Print out the Application Type value counts
APPLICATION_TYPE_Counts = charityDF.APPLICATION_TYPE.value_counts()
APPLICATION_TYPE_Counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Group All Application Types with less than 1000 as "Other"
# Determine which values to replace
replace_type = list(APPLICATION_TYPE_Counts[APPLICATION_TYPE_Counts < 1000].index)

# Replace in DataFrame
for APPLICATION_TYPE in replace_type:
    charityDF.APPLICATION_TYPE = charityDF.APPLICATION_TYPE.replace(APPLICATION_TYPE,"Other")

# Check to make sure binning was successful
charityDF.APPLICATION_TYPE.value_counts()


T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Print out the Application Type value counts
CLASSIFICATION_Counts = charityDF.CLASSIFICATION.value_counts()
CLASSIFICATION_Counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1370        1
C1570        1
C2380        1
C1283        1
C5200        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# Group All CLASSIFICATION Types with less than 1000 as "Other"
# Determine which values to replace
replace_type = list(CLASSIFICATION_Counts[CLASSIFICATION_Counts < 1000].index)

# Replace in DataFrame
for CLASSIFICATION in replace_type:
    charityDF.CLASSIFICATION = charityDF.CLASSIFICATION.replace(CLASSIFICATION,"Other")

# Check to make sure binning was successful
charityDF.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Print out the Application Type value counts
INCOME_AMT_Counts = charityDF.INCOME_AMT.value_counts()
INCOME_AMT_Counts


0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [12]:
# Group All INCOME_AMT Types with non-zero as ">0"  
# Determine which values to replace
replace_type = list(INCOME_AMT_Counts[INCOME_AMT_Counts < 10000].index)

# Replace in DataFrame
for INCOME_AMT in replace_type:
    charityDF.INCOME_AMT = charityDF.INCOME_AMT.replace(INCOME_AMT,">0")

# Check to make sure binning was successful
charityDF.INCOME_AMT.value_counts()

0     24388
>0     9911
Name: INCOME_AMT, dtype: int64

In [13]:
# The large majority of the INCOME AMT are 0, Group all non-zero data together
# Print out the Application Type value counts
AFFILIATION_Counts = charityDF.AFFILIATION.value_counts()
AFFILIATION_Counts

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [14]:
# Group All AFFILIATION Types with less than 100 as "other"  
# Determine which values to replace
replace_type = list(AFFILIATION_Counts[AFFILIATION_Counts < 100].index)

# Replace in DataFrame
for AFFILIATION in replace_type:
    charityDF.AFFILIATION = charityDF.AFFILIATION.replace(AFFILIATION,"Other")

# Check to make sure binning was successful
charityDF.AFFILIATION.value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [15]:
# Print out the Application Type value counts
USE_CASE_Counts = charityDF.USE_CASE.value_counts()
USE_CASE_Counts

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [16]:
# Group USE_CASE Types with less than 1000 as "other"  
# Determine which values to replace
replace_type = list(USE_CASE_Counts[USE_CASE_Counts < 1000].index)

# Replace in DataFrame
for USE_CASE in replace_type:
    charityDF.USE_CASE = charityDF.USE_CASE.replace(USE_CASE,"Other")

# Check to make sure binning was successful
charityDF.USE_CASE.value_counts() 

Preservation    28095
ProductDev       5671
Other             533
Name: USE_CASE, dtype: int64

In [17]:
# Print out the ORGANIZATION Type value counts
ORGANIZATION_Counts = charityDF.ORGANIZATION.value_counts()
ORGANIZATION_Counts

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [18]:
# Group ORGANIZATION Types with less than 1000 as "other"  
# Determine which values to replace
replace_type = list(ORGANIZATION_Counts[ORGANIZATION_Counts < 1000].index)

# Replace in DataFrame
for ORGANIZATION in replace_type:
    charityDF.ORGANIZATION = charityDF.ORGANIZATION.replace(ORGANIZATION,"Other")

# Check to make sure binning was successful
charityDF.ORGANIZATION.value_counts() 

Trust          23515
Association    10255
Other            529
Name: ORGANIZATION, dtype: int64

In [19]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charityDF[charity_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Independent,AFFILIATION_Other,CLASSIFICATION_C1000,...,CLASSIFICATION_C3000,CLASSIFICATION_Other,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Other,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_>0
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [20]:
# Merge one-hot encoded features and drop the originals
charityDF = charityDF.merge(encode_df,left_index=True, right_index=True)
charityDF = charityDF.drop(charity_cat,1)
charityDF.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Independent,...,CLASSIFICATION_C3000,CLASSIFICATION_Other,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Other,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_>0
0,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
3,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
4,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [21]:
# Verify all data in table is numerical
charityDF.dtypes

ASK_AMT                           int64
IS_SUCCESSFUL                     int64
APPLICATION_TYPE_Other          float64
APPLICATION_TYPE_T19            float64
APPLICATION_TYPE_T3             float64
APPLICATION_TYPE_T4             float64
APPLICATION_TYPE_T5             float64
APPLICATION_TYPE_T6             float64
AFFILIATION_CompanySponsored    float64
AFFILIATION_Independent         float64
AFFILIATION_Other               float64
CLASSIFICATION_C1000            float64
CLASSIFICATION_C1200            float64
CLASSIFICATION_C2000            float64
CLASSIFICATION_C2100            float64
CLASSIFICATION_C3000            float64
CLASSIFICATION_Other            float64
USE_CASE_Other                  float64
USE_CASE_Preservation           float64
USE_CASE_ProductDev             float64
ORGANIZATION_Association        float64
ORGANIZATION_Other              float64
ORGANIZATION_Trust              float64
INCOME_AMT_0                    float64
INCOME_AMT_>0                   float64


In [22]:
# Remove IS_SUCCESSFUL target from features data and save as target
y = charityDF.IS_SUCCESSFUL.values
X = charityDF.drop(columns=["IS_SUCCESSFUL"]).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# transform the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
# Create regression model to benchmark Deap learning models

# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.468


In [24]:
# Define the model - deep neural net
number_input_features = len(X_train[0])

hidden_nodes_layer1 =  8  
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 200       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 251
Trainable params: 251
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
25724/25724 [==============================] - 11s 416us/sample - loss: 227.6395 - acc: 0.5065
Epoch 2/100
25724/25724 [==============================] - 9s 362us/sample - loss: 0.6913 - acc: 0.5324
Epoch 3/100
25724/25724 [==============================] - 10s 378us/sample - loss: 0.6911 - acc: 0.5324
Epoch 4/100
25724/25724 [==============================] - 9s 365us/sample - loss: 0.6911 - acc: 0.5324
Epoch 5/100
25724/25724 [==============================] - 9s 354us/sample - loss: 0.6911 - acc: 0.5324
Epoch 6/100
25724/25724 [==============================] - 9s 340us/sample - loss: 0.6911 - acc: 0.5324
Epoch 7/100
25724/25724 [==============================] - 8s 312us/sample - loss: 0.6911 - acc: 0.5324
Epoch 8/100
25724/25724 [==============================] - 9s 343us/sample - loss: 0.6911 - acc: 0.5324
Epoch 9/100
25724/25724 [==============================] - 11s 429us/s

In [26]:
# Define the  2nd model - deep neural net try different activation function
number_input_features = len(X_train[0])

# Results in Accuracy of 0.53
hidden_nodes_layer1 =  8  
hidden_nodes_layer2 = 5

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 200       
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 6         
Total params: 251
Trainable params: 251
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn2.fit(X_train,y_train,epochs=100)
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
25724/25724 [==============================] - 3s 102us/sample - loss: 0.6930 - acc: 0.5267
Epoch 2/100
25724/25724 [==============================] - 3s 101us/sample - loss: 0.6915 - acc: 0.5324
Epoch 3/100
25724/25724 [==============================] - 2s 83us/sample - loss: 0.6913 - acc: 0.5316
Epoch 4/100
25724/25724 [==============================] - 2s 84us/sample - loss: 0.6913 - acc: 0.5324
Epoch 5/100
25724/25724 [==============================] - 3s 106us/sample - loss: 0.6914 - acc: 0.5324
Epoch 6/100
25724/25724 [==============================] - 3s 119us/sample - loss: 0.6913 - acc: 0.5324
Epoch 7/100
25724/25724 [==============================] - 2s 88us/sample - loss: 0.6912 - acc: 0.5314
Epoch 8/100
25724/25724 [==============================] - 2s 86us/sample - loss: 0.6912 - acc: 0.5324
Epoch 9/100
25724/25724 [==============================] - 2s 82us/sample - loss: 0.6915 - acc: 0.5324
Epoch 10/100
25724/25724 [==============================] - 2s 80us/s

KeyboardInterrupt: 

In [28]:
# Define the third model - deep neural net - add a third hidden layer
number_input_features = len(X_train[0])

hidden_nodes_layer1 =  8  
hidden_nodes_layer2 = 6
hidden_nodes_layer3 = 4

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 8)                 200       
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 28        
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 5         
Total params: 287
Trainable params: 287
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn3.fit(X_train,y_train,epochs=100)
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
25724/25724 [==============================] - 2s 65us/sample - loss: 122494.6714 - acc: 0.5324
Epoch 2/100
25724/25724 [==============================] - 2s 68us/sample - loss: 19378.7192 - acc: 0.5248
Epoch 3/100
25724/25724 [==============================] - 2s 84us/sample - loss: 0.8933 - acc: 0.4676
Epoch 4/100
25724/25724 [==============================] - 3s 106us/sample - loss: 0.7667 - acc: 0.4676
Epoch 5/100
25724/25724 [==============================] - 2s 59us/sample - loss: 0.7106 - acc: 0.4676
Epoch 6/100
25724/25724 [==============================] - 1s 56us/sample - loss: 0.6942 - acc: 0.4942
Epoch 7/100
25724/25724 [==============================] - 1s 58us/sample - loss: 0.6913 - acc: 0.5324
Epoch 8/100
25724/25724 [==============================] - 1s 55us/sample - loss: 0.6911 - acc: 0.5324
Epoch 9/100
25724/25724 [==============================] - 2s 60us/sample - loss: 0.6911 - acc: 0.5324
Epoch 10/100
25724/25724 [==============================] - 2s 

In [41]:
# Try 4th model with less variables - Drop ASK_AMT from X_
# X and y have already been created, Drop ASK_AMT and respample training and test data

# Remove IS_SUCCESSFUL and ASK_AMT target from features data and save as target
X = charityDF.drop(columns=["IS_SUCCESSFUL", "ASK_AMT"]).values


# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# transform the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [42]:
# Create regression model to benchmark Deap learning models

# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.713


In [32]:
# Define the model - deep neural net
number_input_features = len(X_train[0])

hidden_nodes_layer1 =  8  
hidden_nodes_layer2 = 5

nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 16)                384       
_________________________________________________________________
dense_11 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 9         
Total params: 529
Trainable params: 529
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn4.fit(X_train,y_train,epochs=100)
# Evaluate the model using the test data
model_loss, model_accuracy = nn4.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
25724/25724 [==============================] - 2s 78us/sample - loss: 0.6029 - acc: 0.6940
Epoch 2/100
25724/25724 [==============================] - 2s 72us/sample - loss: 0.5652 - acc: 0.7239
Epoch 3/100
25724/25724 [==============================] - 2s 64us/sample - loss: 0.5622 - acc: 0.7259
Epoch 4/100
25724/25724 [==============================] - 3s 107us/sample - loss: 0.5606 - acc: 0.7268
Epoch 5/100
25724/25724 [==============================] - 4s 159us/sample - loss: 0.5596 - acc: 0.7268
Epoch 6/100
25724/25724 [==============================] - 4s 143us/sample - loss: 0.5588 - acc: 0.7265
Epoch 7/100
25724/25724 [==============================] - 2s 71us/sample - loss: 0.5580 - acc: 0.7280
Epoch 8/100
25724/25724 [==============================] - 2s 61us/sample - loss: 0.5576 - acc: 0.7268
Epoch 9/100
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5571 - acc: 0.7267
Epoch 10/100
25724/25724 [==============================] - 2s 59us/sa

In [34]:
# Define the model - deep neural net
number_input_features = len(X_train[0])

hidden_nodes_layer1 =  8  
hidden_nodes_layer2 = 5
hidden_nodes_layer3 = 2

nn5 = tf.keras.models.Sequential()

# First hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn5.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn5.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 16)                384       
_________________________________________________________________
dense_14 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_15 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 5         
Total params: 561
Trainable params: 561
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Compile the model
nn5.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn5.fit(X_train,y_train,epochs=100)
# Evaluate the model using the test data
model_loss, model_accuracy = nn5.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
25724/25724 [==============================] - 3s 116us/sample - loss: 0.6080 - acc: 0.6943
Epoch 2/100
25724/25724 [==============================] - 4s 159us/sample - loss: 0.5725 - acc: 0.7262
Epoch 3/100
25724/25724 [==============================] - 4s 155us/sample - loss: 0.5674 - acc: 0.7276
Epoch 4/100
25724/25724 [==============================] - 4s 151us/sample - loss: 0.5647 - acc: 0.7279
Epoch 5/100
25724/25724 [==============================] - 3s 119us/sample - loss: 0.5622 - acc: 0.7274
Epoch 6/100
25724/25724 [==============================] - 3s 127us/sample - loss: 0.5608 - acc: 0.7276
Epoch 7/100
25724/25724 [==============================] - 3s 98us/sample - loss: 0.5605 - acc: 0.7274
Epoch 8/100
25724/25724 [==============================] - 3s 123us/sample - loss: 0.5594 - acc: 0.7269
Epoch 9/100
25724/25724 [==============================] - 3s 105us/sample - loss: 0.5592 - acc: 0.7275
Epoch 10/100
25724/25724 [==============================] - 2s 85